# Chesser

### Initial Setup

In [1]:
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from pathlib import Path

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

# Reproducibility
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Check Tensorflow identifies GPU and enable memory growth
#physical_devices = tf.config.experimental.list_physical_Devices('GPU')
#print("Num GPUs available: " , len(physical_devices))
#tf.config.experimental.set_memory_grows(physical_devices[0], True)


In [2]:
data_dir = Path('input/classes')
image_count = len(list(data_dir.glob('*/*')))
print(image_count)

996


In [3]:
batch_size = 64

# Train Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=123,
  subset="training",
  label_mode='categorical',
  #image_size=(img_height, img_width),
  batch_size=batch_size)

# Validation Dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=123,
  subset="validation",
  label_mode='categorical',
  #image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
num_classes = len(train_ds.class_names)
print(num_classes, class_names)

Found 994 files belonging to 13 classes.
Using 796 files for training.
Found 994 files belonging to 13 classes.
Using 198 files for validation.
13 ['BB', 'BK', 'BN', 'BP', 'BQ', 'BR', 'Empty', 'WB', 'WK', 'WN', 'WP', 'WQ', 'WR']


2022-01-12 07:55:06.931484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 07:55:06.946799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 07:55:06.947457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 07:55:06.948697: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

### Standardize the data

In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

2022-01-12 07:55:08.247950: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


0.12476595 0.8694392


### Configure the dataset for performance

In [5]:
#AUTOTUNE = tf.data.AUTOTUNE

#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
#def configure_for_performance(ds):
#  ds = ds.cache()
#  ds = ds.shuffle(buffer_size=1000)
#  ds = ds.batch(batch_size)
#  ds = ds.prefetch(buffer_size=AUTOTUNE)
#  return ds

#train_ds = configure_for_performance(train_ds)
#val_ds = configure_for_performance(val_ds)

### Train model

In [ ]:
import os
import numpy as np

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras.constraints import max_norm

# architecture of the neural network
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),
                activation='relu',
                kernel_constraint=max_norm(3.)))
model.add(MaxPooling2D(pool_size=(2,2)))
#Conv block 2
model.add(Conv2D(64, kernel_size=(5,5), activation='relu', kernel_constraint=max_norm(3.)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

# Identification
model.add(Flatten())
model.add(Dense(128, activation='relu', name='features'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(13, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='Adam',
             metrics=['accuracy'])
# the loss is minimized by the neural net
# optimizer is the method for training (Adam= stochastic gradient descent, adaptive estimation of first-order and second-order moments)
# metrics accuracy divides total by count, the number of correctly identified images to total number of images

LOGDIR = "./logs"
my_tensorboard = TensorBoard(log_dir = LOGDIR,
                            histogram_freq=1,
                            write_graph=True,
                            write_images=True)
# Hyperparamter
my_epochs = 10

history = model.fit(train_ds,
                   batch_size=batch_size,
                   callbacks=[my_tensorboard],
                   epochs=my_epochs,
                   verbose=1,
                   validation_data=val_ds)

#model.summary()

2022-01-12 07:55:09.473194: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-12 07:55:09.473251: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-01-12 07:55:09.473314: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2022-01-12 07:55:09.782732: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-01-12 07:55:09.786116: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


Epoch 1/10


2022-01-12 07:55:11.942429: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100


 1/13 [=>............................] - ETA: 46s - loss: 13.9167 - accuracy: 0.0312

2022-01-12 07:55:13.726700: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-12 07:55:13.726753: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


...

Epoch 4/10
13/13 [==============================] - 3s 131ms/step - loss: 0.3557 - accuracy: 0.1595 - val_loss: 0.3213 - val_accuracy: 0.0909


2022-01-12 07:55:42.171518: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3657891840 exceeds 10% of free system memory.


Epoch 5/10
13/13 [==============================] - 2s 119ms/step - loss: 0.3293 - accuracy: 0.1683 - val_loss: 0.3028 - val_accuracy: 0.1212


2022-01-12 07:55:50.575575: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3657891840 exceeds 10% of free system memory.


Epoch 6/10
13/13 [==============================] - 3s 127ms/step - loss: 0.2941 - accuracy: 0.2224 - val_loss: 0.2910 - val_accuracy: 0.1364
